In [4]:
import glob
import os
import numpy as np
import cv2
from skimage import feature
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance

## Functions

In [5]:
def ImShowReduced(im, size = (700,700)):
    """Scales an image to a new size and shows it."""
    res = im.copy()
    res.thumbnail(size, Image.ANTIALIAS)
    plt.imshow(res)

In [6]:
def FilterChannel(im, save = False, output = "ImFilterChannel.JPG", show = False):
    """
    Takes an image and return an image of the red channel only.
    Saves it (optional).
    """
    #exif = im.info["exif"]
    res = np.array(im)
    res[::, ::, 1] = 0 # Delete Red Channel
    res[::, ::, 2] = 0 # Delete Red Channel
    res = Image.fromarray(res)
    if save:
        res.save(output, exif=exif)
    if show: 
        ImShowReduced(res)
    return(res)

In [7]:
def EnhanceIm(im, save = False, output = "ImEnchanced.JPG", show = False):
    """
    Takes an image and return an enchanced image with more: sharpness, contrast 
    and a rotation.
    Saves it (optional).
    """
    # Sharpen 
    enhancer = ImageEnhance.Sharpness(im)
    res = enhancer.enhance(8) 
    # Improve contrast
    enhancer = ImageEnhance.Contrast(res)
    res = enhancer.enhance(4)
    # Rotate
    res = res.rotate(90)
    # Save 
    if save:
        res.save(output, exif=exif)
    if show: 
        ImShowReduced(res)
    return(res)

In [8]:
def EdgesDetection(im, save = False, output = "Edges.JPG", show = False ):
    """
    Compute the canny edges detection algo (on the red channel of) an image.
    Returns a binary image of the detected edges.
    """
    # Canny works with grayscale imgs only
    res = np.array(im)[:,:,0]
    edges = feature.canny(res, sigma=3)
    edges = Image.fromarray(edges)
    # Save 
    if save:
        edges.save(output)
    # Show
    if show: 
        ImShowReduced(edges)
    return(edges)

In [9]:
def ImproveEdges(im, save = False, output = "ImproveEdges.JPG", show = False ):
    """
    Naive algo that looks at each column of a boolean image
    and returns an image where only the top element of each column is taken.
    """
    edges = np.array(im)
    new = np.zeros(edges.shape)
    # Iterate through columns
    for icol, col in enumerate(edges.T):
        found = False
        # Iterate though each elt of a col
        for idx in range(len(col)):
            if col[idx]:
                if not found:
                    # If found the top elt of the col
                    # Color it in white
                    new[idx-1:idx+2, icol] = 255
                    found = True
            else:
                # Color it in black
                new[idx, icol] = 0
    # Convert array back to image
    res = Image.fromarray(new).convert('1')
    # Save 
    if save:
        res.save(output)
    # Show
    if show: 
        ImShowReduced(res)
    return(res)

## Process Images

In [10]:
folder = "laser_2411+"
new_folder = "laser_2411edges+"
im_nbr = 100
end_nbr = 101
#end_nbr = im_nbr+1

In [11]:
for image_name in glob.glob(folder+'/*.JPG')[im_nbr:end_nbr]:
    ## Open image
    print(image_name)
    im = Image.open(image_name)
    ## Define saving path
    output = f"{new_folder}/{image_name[len(folder)+1:]}"
    ## Crop 
    res = im.crop((2000, im.size[1]//2, 4500, 2*im.size[1]//3)) 
    ## Remove Red Channel
    #res = FilterChannel(res)
    ## Enhance
    #res = EnhanceIm(res)
    ## Extract Edges
    edges = EdgesDetection(res, False, output, False)
    ## Improve Edges
    edges = ImproveEdges(edges, True, output)

laser_2411+\_DSC3521.JPG
